In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
main_path = '../overfitting/setTraining/select_noOhe/select_softsensor/6var/'
train = np.load(main_path+'train.npy')
test = np.load(main_path+'test.npy')
y_train = np.load(main_path+'y_train.npy')
y_test = np.load(main_path+'y_test.npy')
print('shapes: train = {}, y_train = {}, test = {}, y_teste = {}'.format(train.shape,y_train.shape,test.shape,y_test.shape))

shapes: train = (17389, 6), y_train = (17389, 1), test = (8566, 6), y_teste = (8566, 1)


In [3]:
# standardscaler x data
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_trainScaled = sc_x.fit_transform(train)
X_testScaled = sc_x.fit_transform(test)

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
y = np.ravel(y_train)
y_test = np.ravel(y_test)
print(y.shape)

(17389,)


# Using best hyperparameters
Best parameters found:
 {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 50, 'hidden_layer_sizes': (256, 128, 64, 32), 'learning_rate': 'adaptive', 'power_t': 0.5, 'solver': 'sgd'}

In [5]:
clf = MLPClassifier(solver="sgd",learning_rate="adaptive",hidden_layer_sizes=(256, 128, 64, 32),
batch_size=50,alpha=0.0001,activation="relu",max_iter=500,random_state=1).fit(X_trainScaled, y)
y_pred=clf.predict(X_testScaled)
print(clf.score(X_testScaled, y_test))

0.8918982021947234


C:\Users\Talles\miniconda3\envs\softsensor\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [6]:
from sklearn.metrics import f1_score

# multilabel classification
F1 score reaches its best value at 1 and worst score at 0

F1 = 2 * (precision * recall) / (precision + recall)

In the multi-class and multi-label case, this is the average of the F1 score of each class with weighting depending on the average parameter

### This parameter is required for multiclass/multilabel targets. If None, the scores for each class are returned.
average{‘micro’, ‘macro’, ‘samples’,’weighted’, ‘binary’} or None, default=’binary’

In [7]:
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='weighted'))

# return for each class
print(f1_score(y_test, y_pred, average=None))

0.8924747509872051
0.8918982021947234
0.8920047150455742
[0.86508585 0.84505364 0.86261598 0.91256514 0.97705314]


In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

accuracy_score(y_test, y_pred)

from sklearn.metrics import jaccard_score
jaccard_score(y_test, y_pred, average=None)

array([0.76224784, 0.73168215, 0.75842105, 0.83919063, 0.95513577])

### roc_auc_score
Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.

**multi_class{‘raise’, ‘ovr’, ‘ovo’}, default=’raise’**

Stands for One-vs-rest. Computes the AUC of each class against the rest

In [9]:
y_pred_proba = clf.predict_proba(X_testScaled)
roc_auc_score(y_test, y_pred_proba, average='macro', multi_class='ovr')

0.9878479224917834

### Confusion Matrix

In [10]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1587,   90,   53,   21,    2],
       [ 193, 1418,   45,   45,    6],
       [ 126,   85, 1441,   51,    3],
       [  10,   44,   87, 1576,   15],
       [   0,   12,    9,   29, 1618]], dtype=int64)